In [ ]:
# Import libraries
import numpy as np
import math as math
import pandas as pd
import matplotlib.pyplot as plt
from numpy import loadtxt
from numpy.random import seed

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression, make_blobs
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn import preprocessing, metrics, model_selection

from matplotlib.legend_handler import HandlerLine2D

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import tensorflow.keras.backend as K
from tensorflow.keras.losses import Huber
from keras.models import Sequential, load_model
from keras.layers import Dense

In [ ]:
#Import dataset, PCA analysis, and spilt them:

train_results = []
test_results = []

train_results = []
test_results = []
xls1 = pd.ExcelFile('polymer properties.xls')

file_part1  = pd.DataFrame()

num =0 
while num < 109:
    part1 = pd.read_excel(xls1,'descriptors.xls_'+ str(num))
    file_part1 = pd.concat([file_part1,part1], axis=1)
    num +=1

count =0


dataset = file_part1.values

#name
A  = dataset [:, 1]
#X is the value of descriptors
X = dataset [:,3:]
Y = dataset [:,2]


min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)



# Instantiate a PCA object with 6 components
pca = PCA(n_components=13)

# Fit the PCA model to your scaled dataset
X_pca = pca.fit_transform(X_scale)



#CP:

X_cp= X_pca[:123]
Y_cp=Y[:123]

X_cv= X_pca[123:133]
Y_cv=Y[123:133]
print(Y_cv)

X_flexural= X_pca[133:146]
Y_flexural=Y[133:146]

print(Y_flexural)


X_shear= X_pca[146:164]
Y_shear=Y[146:164]

print(Y_shear)

X_dynamic= X_pca[164:]
Y_dynamic=Y[164:]
print(Y_dynamic)




In [ ]:
#Train model with MAE loss function to predict Cp



# Split the dataset
X_cp_train, X_cp_test, Y_cp_train, Y_cp_test = train_test_split(X_cp, Y_cp, test_size=0.35, random_state=70)

X_cp_train = np.array(X_cp_train).astype("float")
Y_cp_train = np.array(Y_cp_train).astype("float")
X_cp_test = np.array(X_cp_test).astype("float")
Y_cp_test = np.array(Y_cp_test).astype("float")

tf.keras.utils.set_random_seed(20)


# Objective function for Optuna
def objective(trial):

    # Hyperparameters to tune
    num_layers = trial.suggest_int('num_layers', 2, 10)  # Number of layers
    units = trial.suggest_int('units', 32, 1024, step=32)  # Number of neurons in each layer
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)  # Dropout rate
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)  # Learning rate

    # Build model based on suggested parameters
    model = keras.Sequential()
    model.add(tf.keras.layers.Dense(units, activation='relu', input_dim=13))
    
    for i in range(num_layers):
        model.add(tf.keras.layers.Dense(units, activation='relu'))
        model.add(tf.keras.layers.Dropout(dropout_rate))
    
    model.add(tf.keras.layers.Dense(1, activation='linear'))  # Output layer

    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mae', metrics=['accuracy'])

    # Train the model
    model.fit(X_cp_train, Y_cp_train, validation_split=0.3, epochs=trial.suggest_int('epochs', 50, 300), batch_size=10, verbose=0)

    # Make predictions
    Y_cp_pred_test = model.predict(X_cp_test)

    # Compute mean absolute error (MAE) as the objective metric
    mae_test = mean_absolute_error(Y_cp_test, Y_cp_pred_test)
    
    return mae_test  # Optuna will minimize this value


# Create an Optuna study and optimize
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Print the best trial results
print("Best trial:")
trial = study.best_trial
print("  Value (MAE):", trial.value)
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Plot results of the best model
best_model = keras.Sequential()
best_model.add(tf.keras.layers.Dense(trial.params['units'], activation='relu', input_dim=13))

for i in range(trial.params['num_layers']):
    best_model.add(tf.keras.layers.Dense(trial.params['units'], activation='relu'))
    best_model.add(tf.keras.layers.Dropout(trial.params['dropout_rate']))

best_model.add(tf.keras.layers.Dense(1, activation='linear'))

# Compile the best model
optimizer = keras.optimizers.Adam(learning_rate=trial.params['learning_rate'])
best_model.compile(optimizer=optimizer, loss='mae', metrics=['accuracy'])

best_model.fit(X_cp_train, Y_cp_train, validation_split=0.3, epochs=trial.params['epochs'], batch_size=10)

Y_cp_pred_train = best_model.predict(X_cp_train)
Y_cp_pred_test = best_model.predict(X_cp_test)

# R2 Score
R2train = r2_score(Y_cp_train, Y_cp_pred_train)
R2test = r2_score(Y_cp_test, Y_cp_pred_test)

print('R2 Score on Train set:', R2train)
print('R2 Score on Test set:', R2test)

# Plot actual vs predicted
plt.figure(dpi=200)
x = [0, 500]
y = [0, 500]
plt.plot(x, y, '--', color='k')
plt.xlim(0, 500)
plt.ylim(0, 500)
plt.plot(Y_cp_train, Y_cp_pred_train, 'o', color='c', label="Training set")
plt.plot(Y_cp_test, Y_cp_pred_test, 'o', color='k', label="Test set")
plt.xlabel('Expected $C_{p}$ (cal/g.C)', labelpad=12, fontsize=22, fontname='Times New Roman')
plt.ylabel('Predicted $C_{p}$ (cal/g.C)', labelpad=12, fontsize=22, fontname='Times New Roman')
plt.legend()
plt.show()



mse_train = mean_squared_error(Y_cp_train, Y_cp_pred_train)
mse_test = mean_squared_error(Y_cp_test, Y_cp_pred_test)

print('MSE on train set:', mse_train)
print('MSE on test set:', mse_test)


mse_per_sample_train = mse_train / len(Y_cp_train)
mse_per_sample_test = mse_test / len(Y_cp_test)

print('MSE per sample on train set:', mse_per_sample_train)
print('MSE per sample on test set:', mse_per_sample_test)




mae_train = mean_absolute_error(Y_cp_train, Y_cp_pred_train)
mae_test = mean_absolute_error(Y_cp_test, Y_cp_pred_test)

print('MAE on train set:', mae_train)
print('MAE on test set:', mae_test)

mae_per_sample_train = mae_train / len(Y_cp_train)
mae_per_sample_test = mae_test / len(Y_cp_test)

print('MAE per sample on train set:', mae_per_sample_train)
print('MAE per sample on test set:', mae_per_sample_test)




def huber_loss(y_true, y_pred, delta=1.0):
    #loss = tf.keras.losses.Huber(delta)(y_true, y_pred)
    loss = Huber(delta)(y_true, y_pred)
    return loss.numpy()

huber_loss_train = huber_loss(Y_cp_train, Y_cp_pred_train)
huber_loss_test = huber_loss(Y_cp_test, Y_cp_pred_test)


print('Huber Loss on train set:', huber_loss_train)
print('Huber Loss on test set:', huber_loss_test)



def wing_loss(y_true, y_pred, w=5.0, epsilon=1.5):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    w = tf.cast(w, tf.float64)
    epsilon = tf.cast(epsilon, tf.float64)
    c = w * (1.0 - K.log(1.0 + w/epsilon))
    absolute = K.abs(y_true - y_pred)
    losses = tf.where(absolute < w, w * K.log(2.0 + absolute/epsilon), absolute - c)
    return K.mean(losses).numpy()


wing_loss_train = wing_loss(Y_cp_train, Y_cp_pred_train)
wing_loss_test = wing_loss(Y_cp_test, Y_cp_pred_test)

print('Wing Loss on train set:', wing_loss_train)
print('Wing Loss on test set:', wing_loss_test)



In [ ]:
##Train model with MSE loss function to predict Cp

# Split the dataset
X_cp_train, X_cp_test, Y_cp_train, Y_cp_test = train_test_split(X_cp, Y_cp, test_size=0.35, random_state=70)

X_cp_train = np.array(X_cp_train).astype("float")
Y_cp_train = np.array(Y_cp_train).astype("float")
X_cp_test = np.array(X_cp_test).astype("float")
Y_cp_test = np.array(Y_cp_test).astype("float")

tf.keras.utils.set_random_seed(20)

# Optuna objective function
def objective(trial):
    # Hyperparameters to tune
    num_layers = trial.suggest_int('num_layers', 2, 10)  # Number of hidden layers
    units = trial.suggest_int('units', 128, 1024, step=64)  # Number of neurons in each layer
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)  # Dropout rate
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)  # Learning rate
    batch_size = trial.suggest_int('batch_size', 10, 100, step=10)  # Batch size
    epochs = trial.suggest_int('epochs', 100, 700)  # Epochs

    # Model architecture
    model = keras.Sequential()
    model.add(layers.Dense(units, activation='relu', input_dim=13))
    
    for i in range(num_layers):
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.Dropout(dropout_rate))  # Dropout
    
    model.add(layers.Dense(1, activation='linear'))  # Output layer

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='mse', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(X_cp_train, Y_cp_train, validation_split=0.3, epochs=epochs, batch_size=batch_size, verbose=0)

    # Predict on test set
    Y_cp_pred_test = model.predict(X_cp_test)
    
    # Calculate R2 score as the evaluation metric (the higher, the better)
    R2test = r2_score(Y_cp_test, Y_cp_pred_test)
    
    # Since Optuna minimizes the objective function, return negative R2
    return -R2test

# Create Optuna study and run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Print the best trial results
print('Best trial:')
trial = study.best_trial
print('  Value:', -trial.value)  # Convert back to positive R2
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

# Plot the training and validation loss over epochs for the best model
best_model = keras.Sequential()
best_model.add(layers.Dense(trial.params['units'], activation='relu', input_dim=13))

for i in range(trial.params['num_layers']):
    best_model.add(layers.Dense(trial.params['units'], activation='relu'))
    best_model.add(layers.Dropout(trial.params['dropout_rate']))

best_model.add(layers.Dense(1, activation='linear'))

# Compile the best model
best_model.compile(optimizer=keras.optimizers.Adam(learning_rate=trial.params['learning_rate']), loss='mse', metrics=['accuracy'])

# Train the best model
history_callback = best_model.fit(X_cp_train, Y_cp_train, validation_split=0.3, epochs=trial.params['epochs'], batch_size=trial.params['batch_size'])

# Predict on training and test sets
Y_cp_pred_train = best_model.predict(X_cp_train)
Y_cp_pred_test = best_model.predict(X_cp_test)

# R2 Scores
R2train = r2_score(Y_cp_train, Y_cp_pred_train)
R2test = r2_score(Y_cp_test, Y_cp_pred_test)

print('R2 Score on Train set:', R2train)
print('R2 Score on Test set:', R2test)

# Plot training and validation loss over epochs
history_dict = history_callback.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
fig = plt.figure(dpi=200)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Plot actual vs predicted
fig = plt.figure(dpi=200)
x = [0, 500]
y = [0, 500]
plt.plot(x, y, '--', color='k')
plt.xlim(0, 500)
plt.ylim(0, 500)
plt.plot(Y_cp_train, Y_cp_pred_train, 'o', color='#ADFF2F', label="Training set")
plt.plot(Y_cp_test, Y_cp_pred_test, 'o', color='k', label="Test set")
plt.xlabel('Expected $C_{p}$ (cal/g.C)', labelpad=12, fontsize=22, fontname='Times New Roman')
plt.ylabel('Predicted $C_{p}$ (cal/g.C)', labelpad=12, fontsize=22, fontname='Times New Roman')
plt.legend()
plt.show()


#save the model to use in Transfer learning

model.save('Model.h5')




mse_train = mean_squared_error(Y_cp_train, Y_cp_pred_train)
mse_test = mean_squared_error(Y_cp_test, Y_cp_pred_test)

print('MSE on train set:', mse_train)
print('MSE on test set:', mse_test)


mse_per_sample_train = mse_train / len(Y_cp_train)
mse_per_sample_test = mse_test / len(Y_cp_test)

print('MSE per sample on train set:', mse_per_sample_train)
print('MSE per sample on test set:', mse_per_sample_test)




mae_train = mean_absolute_error(Y_cp_train, Y_cp_pred_train)
mae_test = mean_absolute_error(Y_cp_test, Y_cp_pred_test)

print('MAE on train set:', mae_train)
print('MAE on test set:', mae_test)

mae_per_sample_train = mae_train / len(Y_cp_train)
mae_per_sample_test = mae_test / len(Y_cp_test)

print('MAE per sample on train set:', mae_per_sample_train)
print('MAE per sample on test set:', mae_per_sample_test)




def huber_loss(y_true, y_pred, delta=1.0):
    #loss = tf.keras.losses.Huber(delta)(y_true, y_pred)
    loss = Huber(delta)(y_true, y_pred)
    return loss.numpy()

huber_loss_train = huber_loss(Y_cp_train, Y_cp_pred_train)
huber_loss_test = huber_loss(Y_cp_test, Y_cp_pred_test)


print('Huber Loss on train set:', huber_loss_train)
print('Huber Loss on test set:', huber_loss_test)





def wing_loss(y_true, y_pred, w=5.0, epsilon=1.5):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    w = tf.cast(w, tf.float64)
    epsilon = tf.cast(epsilon, tf.float64)
    c = w * (1.0 - K.log(1.0 + w/epsilon))
    absolute = K.abs(y_true - y_pred)
    losses = tf.where(absolute < w, w * K.log(2.0 + absolute/epsilon), absolute - c)
    return K.mean(losses).numpy()


wing_loss_train = wing_loss(Y_cp_train, Y_cp_pred_train)
wing_loss_test = wing_loss(Y_cp_test, Y_cp_pred_test)

print('Wing Loss on train set:', wing_loss_train)
print('Wing Loss on test set:', wing_loss_test)



In [ ]:
#Train model with Huber loss function to predict Cp

# Split the dataset
X_cp_train, X_cp_test, Y_cp_train, Y_cp_test = train_test_split(X_cp, Y_cp, test_size=0.35, random_state=70)

X_cp_train = np.array(X_cp_train).astype("float")
Y_cp_train = np.array(Y_cp_train).astype("float")
X_cp_test = np.array(X_cp_test).astype("float")
Y_cp_test = np.array(Y_cp_test).astype("float")

tf.keras.utils.set_random_seed(20)

# Optuna objective function
def objective(trial):
    # Hyperparameters to tune
    num_layers = trial.suggest_int('num_layers', 2, 10)  # Number of hidden layers
    units = trial.suggest_int('units', 128, 1024, step=64)  # Number of neurons in each layer
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)  # Dropout rate
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)  # Learning rate
    batch_size = trial.suggest_int('batch_size', 10, 100, step=10)  # Batch size
    epochs = trial.suggest_int('epochs', 100, 700)  # Epochs

    # Model architecture
    model = keras.Sequential()
    model.add(layers.Dense(units, activation='relu', input_dim=13))
    
    for _ in range(num_layers - 1):  # Add the specified number of layers
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.Dropout(dropout_rate))  # Dropout
    
    model.add(layers.Dense(1, activation='linear'))  # Output layer

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
                  loss=tf.keras.losses.Huber(delta=1.0), 
                  metrics=['accuracy'])
    
    # Train the model
    history = model.fit(X_cp_train, Y_cp_train, validation_split=0.3, epochs=epochs, 
                        batch_size=batch_size, verbose=0)

    # Predict on test set
    Y_cp_pred_test = model.predict(X_cp_test)
    
    # Calculate R2 score as the evaluation metric (the higher, the better)
    R2test = r2_score(Y_cp_test, Y_cp_pred_test)
    
    # Since Optuna minimizes the objective function, return negative R2
    return -R2test

# Create Optuna study and run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Print the best trial results
print('Best trial:')
trial = study.best_trial
print('  Value:', -trial.value)  # Convert back to positive R2
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

# Train the best model with the optimized hyperparameters
best_model = keras.Sequential()
best_model.add(layers.Dense(trial.params['units'], activation='relu', input_dim=13))

for _ in range(trial.params['num_layers'] - 1):
    best_model.add(layers.Dense(trial.params['units'], activation='relu'))
    best_model.add(layers.Dropout(trial.params['dropout_rate']))

best_model.add(layers.Dense(1, activation='linear'))  # Output layer

# Compile the best model
best_model.compile(optimizer=keras.optimizers.Adam(learning_rate=trial.params['learning_rate']),
                   loss=tf.keras.losses.Huber(delta=1.0), 
                   metrics=['accuracy'])

# Train the best model
history_callback = best_model.fit(X_cp_train, Y_cp_train, validation_split=0.3, 
                                   epochs=trial.params['epochs'], 
                                   batch_size=trial.params['batch_size'])

# Predict on training and test sets
Y_cp_pred_train = best_model.predict(X_cp_train)
Y_cp_pred_test = best_model.predict(X_cp_test)

# R2 Scores
R2train = r2_score(Y_cp_train, Y_cp_pred_train)
R2test = r2_score(Y_cp_test, Y_cp_pred_test)

print('R2 Score on Train set:', R2train)
print('R2 Score on Test set:', R2test)

# Plot training and validation loss over epochs
history_dict = history_callback.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
fig = plt.figure(dpi=200)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot actual vs predicted
fig = plt.figure(dpi=200)
x = [0, 500]
y = [0, 500]
plt.plot(x, y, '--', color='k')
plt.xlim(0, 500)
plt.ylim(0, 500)
plt.plot(Y_cp_train, Y_cp_pred_train, 'o', color='#ADFF2F', label="Training set")
plt.plot(Y_cp_test, Y_cp_pred_test, 'o', color='k', label="Test set")
plt.xlabel('Expected $C_{p}$ (cal/g.C)', labelpad=12, fontsize=22, fontname='Times New Roman')
plt.ylabel('Predicted $C_{p}$ (cal/g.C)', labelpad=12, fontsize=22, fontname='Times New Roman')
plt.legend()
plt.show()



mse_train = mean_squared_error(Y_cp_train, Y_cp_pred_train)
mse_test = mean_squared_error(Y_cp_test, Y_cp_pred_test)

print('MSE on train set:', mse_train)
print('MSE on test set:', mse_test)


mse_per_sample_train = mse_train / len(Y_cp_train)
mse_per_sample_test = mse_test / len(Y_cp_test)

print('MSE per sample on train set:', mse_per_sample_train)
print('MSE per sample on test set:', mse_per_sample_test)




mae_train = mean_absolute_error(Y_cp_train, Y_cp_pred_train)
mae_test = mean_absolute_error(Y_cp_test, Y_cp_pred_test)

print('MAE on train set:', mae_train)
print('MAE on test set:', mae_test)

mae_per_sample_train = mae_train / len(Y_cp_train)
mae_per_sample_test = mae_test / len(Y_cp_test)

print('MAE per sample on train set:', mae_per_sample_train)
print('MAE per sample on test set:', mae_per_sample_test)




def huber_loss(y_true, y_pred, delta=1.0):
    #loss = tf.keras.losses.Huber(delta)(y_true, y_pred)
    loss = Huber(delta)(y_true, y_pred)
    return loss.numpy()

huber_loss_train = huber_loss(Y_cp_train, Y_cp_pred_train)
huber_loss_test = huber_loss(Y_cp_test, Y_cp_pred_test)


print('Huber Loss on train set:', huber_loss_train)
print('Huber Loss on test set:', huber_loss_test)





def wing_loss(y_true, y_pred, w=5.0, epsilon=1.5):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    w = tf.cast(w, tf.float64)
    epsilon = tf.cast(epsilon, tf.float64)
    c = w * (1.0 - K.log(1.0 + w/epsilon))
    absolute = K.abs(y_true - y_pred)
    losses = tf.where(absolute < w, w * K.log(2.0 + absolute/epsilon), absolute - c)
    return K.mean(losses).numpy()


wing_loss_train = wing_loss(Y_cp_train, Y_cp_pred_train)
wing_loss_test = wing_loss(Y_cp_test, Y_cp_pred_test)

print('Wing Loss on train set:', wing_loss_train)
print('Wing Loss on test set:', wing_loss_test)

In [ ]:
#Train model with wing shape function to predict Cp


# Split the dataset (ensure X_cp and Y_cp are defined)
X_cp_train, X_cp_test, Y_cp_train, Y_cp_test = train_test_split(X_cp, Y_cp, test_size=0.35, random_state=70)

X_cp_train = np.array(X_cp_train).astype("float")
Y_cp_train = np.array(Y_cp_train).astype("float")
X_cp_test = np.array(X_cp_test).astype("float")
Y_cp_test = np.array(Y_cp_test).astype("float")

tf.keras.utils.set_random_seed(20)

# Define Huber loss function
def huber_loss(y_true, y_pred, delta=1.0):
    loss = tf.keras.losses.Huber(delta)(y_true, y_pred)
    return loss.numpy()

# Define Wing loss function
def wing_loss(y_true, y_pred, w=5.0, epsilon=1.5):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    w = tf.cast(w, tf.float64)
    epsilon = tf.cast(epsilon, tf.float64)
    c = w * (1.0 - tf.math.log(1.0 + w/epsilon))
    absolute = tf.abs(y_true - y_pred)
    losses = tf.where(absolute < w, w * tf.math.log(2.0 + absolute/epsilon), absolute - c)
    return tf.reduce_mean(losses).numpy()

# Optuna objective function
def objective(trial):
    # Hyperparameters to tune
    num_layers = trial.suggest_int('num_layers', 2, 10)  # Number of hidden layers
    units = trial.suggest_int('units', 128, 1024, step=64)  # Number of neurons in each layer
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)  # Dropout rate
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)  # Learning rate
    batch_size = trial.suggest_int('batch_size', 10, 100, step=10)  # Batch size
    epochs = trial.suggest_int('epochs', 100, 700)  # Epochs

    # Model architecture
    model = keras.Sequential()
    model.add(layers.Dense(units, activation='relu', input_dim=13))
    
    for _ in range(num_layers - 1):  # Add the specified number of layers
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.Dropout(dropout_rate))  # Dropout
    
    model.add(layers.Dense(1, activation='linear'))  # Output layer

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
                  loss=wing_loss, 
                  metrics=['accuracy'])
    
    # Train the model
    model.fit(X_cp_train, Y_cp_train, validation_split=0.3, epochs=epochs, 
              batch_size=batch_size, verbose=0)

    # Predict on test set
    Y_cp_pred_test = model.predict(X_cp_test)
    
    # Calculate R2 score as the evaluation metric (the higher, the better)
    R2test = r2_score(Y_cp_test, Y_cp_pred_test)
    
    # Since Optuna minimizes the objective function, return negative R2
    return -R2test

# Create Optuna study and run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Print the best trial results
print('Best trial:')
trial = study.best_trial
print('  Value:', -trial.value)  # Convert back to positive R2
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

# Train the best model with the optimized hyperparameters
best_model = keras.Sequential()
best_model.add(layers.Dense(trial.params['units'], activation='relu', input_dim=13))

for _ in range(trial.params['num_layers'] - 1):
    best_model.add(layers.Dense(trial.params['units'], activation='relu'))
    best_model.add(layers.Dropout(trial.params['dropout_rate']))

best_model.add(layers.Dense(1, activation='linear'))  # Output layer

# Compile the best model
best_model.compile(optimizer=keras.optimizers.Adam(learning_rate=trial.params['learning_rate']),
                   loss=wing_loss, 
                   metrics=['accuracy'])

# Train the best model
history_callback = best_model.fit(X_cp_train, Y_cp_train, validation_split=0.3, 
                                   epochs=trial.params['epochs'], 
                                   batch_size=trial.params['batch_size'])

# Predict on training and test sets
Y_cp_pred_train = best_model.predict(X_cp_train)
Y_cp_pred_test = best_model.predict(X_cp_test)

# R2 Scores
R2train = r2_score(Y_cp_train, Y_cp_pred_train)
R2test = r2_score(Y_cp_test, Y_cp_pred_test)

print('R2 Score on Train set:', R2train)
print('R2 Score on Test set:', R2test)

# Mean Squared Error (MSE)
mse_train = mean_squared_error(Y_cp_train, Y_cp_pred_train)
mse_test = mean_squared_error(Y_cp_test, Y_cp_pred_test)

print('MSE on train set:', mse_train)
print('MSE on test set:', mse_test)

mse_per_sample_train = mse_train / len(Y_cp_train)
mse_per_sample_test = mse_test / len(Y_cp_test)

print('MSE per sample on train set:', mse_per_sample_train)
print('MSE per sample on test set:', mse_per_sample_test)

# Mean Absolute Error (MAE)
mae_train = mean_absolute_error(Y_cp_train, Y_cp_pred_train)
mae_test = mean_absolute_error(Y_cp_test, Y_cp_pred_test)

print('MAE on train set:', mae_train)
print('MAE on test set:', mae_test)

mae_per_sample_train = mae_train / len(Y_cp_train)
mae_per_sample_test = mae_test / len(Y_cp_test)

print('MAE per sample on train set:', mae_per_sample_train)
print('MAE per sample on test set:', mae_per_sample_test)

# Plot training and validation loss over epochs
history_dict = history_callback.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
fig = plt.figure(dpi=200)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot actual vs predicted
fig = plt.figure(dpi=200)
x = [0, 500]
y = [0, 500]
plt.plot(x, y, '--', color='k')
plt.xlim(0, 500)
plt.ylim(0, 500)
plt.plot(Y_cp_train, Y_cp_pred_train, 'o', color='#ADFF2F', label="Training set")
plt.plot(Y_cp_test, Y_cp_pred_test, 'o', color='k', label="Test set")
plt.xlabel('Expected $C_{p}$ (cal/g.C)', labelpad=12, fontsize=22, fontname='Times New Roman')
plt.ylabel('Predicted $C_{p}$ (cal/g.C)', labelpad=12, fontsize=22, fontname='Times New Roman')
plt.legend(loc='best', fontsize=7, prop={'family': 'Times New Roman'})
plt.show()


mse_train = mean_squared_error(Y_cp_train, Y_cp_pred_train)
mse_test = mean_squared_error(Y_cp_test, Y_cp_pred_test)

print('MSE on train set:', mse_train)
print('MSE on test set:', mse_test)


mse_per_sample_train = mse_train / len(Y_cp_train)
mse_per_sample_test = mse_test / len(Y_cp_test)

print('MSE per sample on train set:', mse_per_sample_train)
print('MSE per sample on test set:', mse_per_sample_test)




mae_train = mean_absolute_error(Y_cp_train, Y_cp_pred_train)
mae_test = mean_absolute_error(Y_cp_test, Y_cp_pred_test)

print('MAE on train set:', mae_train)
print('MAE on test set:', mae_test)

mae_per_sample_train = mae_train / len(Y_cp_train)
mae_per_sample_test = mae_test / len(Y_cp_test)

print('MAE per sample on train set:', mae_per_sample_train)
print('MAE per sample on test set:', mae_per_sample_test)




def huber_loss(y_true, y_pred, delta=1.0):
    #loss = tf.keras.losses.Huber(delta)(y_true, y_pred)
    loss = Huber(delta)(y_true, y_pred)
    return loss.numpy()

huber_loss_train = huber_loss(Y_cp_train, Y_cp_pred_train)
huber_loss_test = huber_loss(Y_cp_test, Y_cp_pred_test)


print('Huber Loss on train set:', huber_loss_train)
print('Huber Loss on test set:', huber_loss_test)





def wing_loss(y_true, y_pred, w=5.0, epsilon=1.5):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    w = tf.cast(w, tf.float64)
    epsilon = tf.cast(epsilon, tf.float64)
    c = w * (1.0 - K.log(1.0 + w/epsilon))
    absolute = K.abs(y_true - y_pred)
    losses = tf.where(absolute < w, w * K.log(2.0 + absolute/epsilon), absolute - c)
    return K.mean(losses).numpy()


wing_loss_train = wing_loss(Y_cp_train, Y_cp_pred_train)
wing_loss_test = wing_loss(Y_cp_test, Y_cp_pred_test)

print('Wing Loss on train set:', wing_loss_train)
print('Wing Loss on test set:', wing_loss_test)

In [ ]:
#Train model with combined (MSE,MAE, wing shape, and Huber)  loss function to predict Cp
# Split the dataset (ensure X_cp and Y_cp are defined)
X_cp_train, X_cp_test, Y_cp_train, Y_cp_test = train_test_split(X_cp, Y_cp, test_size=0.35, random_state=70)

X_cp_train = np.array(X_cp_train).astype("float")
Y_cp_train = np.array(Y_cp_train).astype("float")
X_cp_test = np.array(X_cp_test).astype("float")
Y_cp_test = np.array(Y_cp_test).astype("float")

tf.keras.utils.set_random_seed(20)

# Define the custom combined loss function
def combined_loss(y_true, y_pred):
    # Define the parameters for the loss functions
    delta = 1
    epsilon = 1.5
    w = 5

    # Compute the Mean Squared Error loss
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)

    # Compute the Mean Absolute Error loss
    mae_loss = tf.keras.losses.mean_absolute_error(y_true, y_pred)

    # Compute the Huber loss
    huber_loss = tf.keras.losses.Huber(delta)(y_true, y_pred)

    # Compute the Wing loss
    residuals = tf.abs(y_true - y_pred)
    wing_loss = tf.reduce_mean(
        tf.where(residuals < w,
                 w * tf.math.log(1.0 + (residuals / w)),
                 residuals - (w * (tf.math.log(1.0 + (w / epsilon))))
                )
    )

    # Combine the losses using a weighted sum
    loss = (0.25 * mse_loss) + (0.25 * huber_loss) + (0.25 * wing_loss) + (0.25 * mae_loss)

    return loss

# Define the model creation function
def create_model(trial):
    model = keras.Sequential()
    
    # Hyperparameters to optimize
    model.add(layers.Dense(trial.suggest_int('units_1', 128, 1024), activation='relu', input_dim=13))
    model.add(layers.BatchNormalization())
    
    for i in range(trial.suggest_int('n_layers', 1, 5)):
        model.add(layers.Dense(trial.suggest_int(f'units_{i+2}', 128, 1024), activation='relu'))

    model.add(layers.Dense(1, activation='linear'))  # Output layer

    # Suggest a learning rate
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss=combined_loss, metrics=['mae'])
    return model

# Define the objective function for Optuna
def objective(trial):
  
    # Create model
    model = create_model(trial)

    # Train the model
    history = model.fit(X_cp_train, Y_cp_train, validation_split=0.3, 
                        epochs=100, batch_size=10, verbose=0)

    # Evaluate model performance
    Y_cp_pred_test = model.predict(X_cp_test)
    r2 = r2_score(Y_cp_test, Y_cp_pred_test)
    return r2  # Return R^2 score for optimization

# Create a study object and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Change the number of trials as needed

# Print the best hyperparameters found
print("Best hyperparameters: ", study.best_params)

# Optionally, you can train the model with the best parameters and visualize the results
best_model = create_model(study.best_trial)
best_model.fit(X_cp_train, Y_cp_train, validation_split=0.3, epochs=100, batch_size=10, verbose=1)

# Predict and plot results
Y_cp_pred_train = best_model.predict(X_cp_train)
Y_cp_pred_test = best_model.predict(X_cp_test)

# Calculate R² scores
R2train = r2_score(Y_cp_train, Y_cp_pred_train)
R2test = r2_score(Y_cp_test, Y_cp_pred_test)

print('R² score on training set:', R2train)
print('R² score on test set:', R2test)

# Plotting actual vs predicted values
fig = plt.figure(dpi=200)
x = [0, 500]
y = [0, 500]
plt.plot(x, y, '--', color='k')
plt.xlim(0, 500)
plt.ylim(0, 500)
plt.plot(Y_cp_train, Y_cp_pred_train, 'o', color='c', label="Training set")
plt.plot(Y_cp_test, Y_cp_pred_test, 'o', color='k', label="Test set")
plt.xlabel('Expected $C_{p}$ (cal/g.C)', labelpad=12, fontsize=22, fontname='Times New Roman')
plt.ylabel('Predicted $C_{p}$ (cal/g.C)', labelpad=12, fontsize=22, fontname='Times New Roman')
plt.legend(loc='best', fontsize=7, prop={'family': 'Times New Roman'})
plt.title('Expected vs Predicted $C_{p}$', fontsize=22)
plt.grid()
plt.show()



mse_train = mean_squared_error(Y_cp_train, Y_cp_pred_train)
mse_test = mean_squared_error(Y_cp_test, Y_cp_pred_test)

print('MSE on train set:', mse_train)
print('MSE on test set:', mse_test)


mse_per_sample_train = mse_train / len(Y_cp_train)
mse_per_sample_test = mse_test / len(Y_cp_test)

print('MSE per sample on train set:', mse_per_sample_train)
print('MSE per sample on test set:', mse_per_sample_test)




mae_train = mean_absolute_error(Y_cp_train, Y_cp_pred_train)
mae_test = mean_absolute_error(Y_cp_test, Y_cp_pred_test)

print('MAE on train set:', mae_train)
print('MAE on test set:', mae_test)

mae_per_sample_train = mae_train / len(Y_cp_train)
mae_per_sample_test = mae_test / len(Y_cp_test)

print('MAE per sample on train set:', mae_per_sample_train)
print('MAE per sample on test set:', mae_per_sample_test)




def huber_loss(y_true, y_pred, delta=1.0):
    #loss = tf.keras.losses.Huber(delta)(y_true, y_pred)
    loss = Huber(delta)(y_true, y_pred)
    return loss.numpy()

huber_loss_train = huber_loss(Y_cp_train, Y_cp_pred_train)
huber_loss_test = huber_loss(Y_cp_test, Y_cp_pred_test)


print('Huber Loss on train set:', huber_loss_train)
print('Huber Loss on test set:', huber_loss_test)





def wing_loss(y_true, y_pred, w=5.0, epsilon=1.5):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    w = tf.cast(w, tf.float64)
    epsilon = tf.cast(epsilon, tf.float64)
    c = w * (1.0 - K.log(1.0 + w/epsilon))
    absolute = K.abs(y_true - y_pred)
    losses = tf.where(absolute < w, w * K.log(2.0 + absolute/epsilon), absolute - c)
    return K.mean(losses).numpy()


wing_loss_train = wing_loss(Y_cp_train, Y_cp_pred_train)
wing_loss_test = wing_loss(Y_cp_test, Y_cp_pred_test)

print('Wing Loss on train set:', wing_loss_train)
print('Wing Loss on test set:', wing_loss_test)
